# How to combine workgraphs
## Introduction
There are two ways to combine workgraphs:
- Use the graph builder inside another workgraph. 
- Add the workgraph directly to another workgraph.

This tutorial will show an example for the second case. You need to install `workgraph_collections` to run this tutorial. You can install it by running `pip install workgraph_collections`.



In [1]:
%load_ext aiida
from aiida import load_profile
load_profile()

Profile<uuid='57ccbf7d9e2b41b39edb2bfdaf725feb' name='default'>

## Relax, Bands and PDOS
Here is an example to combine `bands` workgraph, `pdos` workgraph with a `relax` node to form a new workgraph.

### Bands workgraph

In [2]:
from workgraph_collections.qe.bands import bands_workgraph
from aiida import load_profile
load_profile()

bands_wg = bands_workgraph(run_relax=False)
bands_wg.to_html()

### PDOS workgraph

In [3]:
from workgraph_collections.qe.pdos import pdos_workgraph
pdos_wg = pdos_workgraph(run_scf=True)
pdos_wg.to_html()

### Extend workgraph

First, create a workgraph with `relax` node:

In [5]:
from aiida_workgraph import WorkGraph
from aiida_quantumespresso.workflows.pw.relax import PwRelaxWorkChain

wg = WorkGraph('Electronic Structure')
relax_node = wg.nodes.new(PwRelaxWorkChain, name='relax')
wg

NodeGraphWidget(settings={'minimap': True}, style={'width': '90%', 'height': '600px'}, value={'name': 'Electro…

Now let's combine `bands` workgraph, `pdos` workgraph with a `relax` node to form a new workgraph.

To add a workgraph into another workgraph, there are two steps:
- extend the workgraph
- adjust the links, add new links between nodes, and remove old links if needed.



In [6]:
# extend a wroktree
wg.extend(bands_wg, prefix='bands_')
wg.extend(pdos_wg, prefix='pdos_')
# adjust the links
wg.links.new(relax_node.outputs['output_structure'], wg.nodes['bands_scf'].inputs['pw.structure'])
wg.links.new(relax_node.outputs['output_structure'], wg.nodes['pdos_scf'].inputs['pw.structure'])
wg.to_html()

## Graph builder
One can compare it with the other method: using the workgraph as a node inside another workgraph.



In [7]:
wg = WorkGraph('Electronic Structure')
relax_node = wg.nodes.new(PwRelaxWorkChain, name='relax')
bands_job = wg.nodes.new(bands_workgraph, name='bands_group')
pdos_job = wg.nodes.new(pdos_workgraph, name='pdos_group')
wg.links.new(relax_node.outputs['output_structure'], bands_job.inputs['structure'])
wg.links.new(relax_node.outputs['output_structure'], pdos_job.inputs['structure'])
wg.to_html()

### Pros
Show the top level workflow.
### Cons
Only know the exact node graph at runtime.